In [3]:
import os
import openpyxl
import re

import pandas as pd
from uk import extract_and_save_uk,create_new_sheet_uk
from german import extract_and_save_german,create_new_sheet_german
from spain import extract_and_save_spain,create_new_sheet_spain
from france import extract_and_save_france,create_new_sheet_france
# from italy import extract_and_save_italy

from glovar import IPHONE_5G_LIST

def load_map_dict_from_execl(dic_excel_path):
    mapbook = openpyxl.load_workbook(dic_excel_path)
    sheet = mapbook.sheetnames[0]
    map_dic = {}
    for i,row in enumerate(mapbook[sheet].values):
        phone_name_before,phone_name_after = row[0].strip(),row[1].strip()
        map_dic[phone_name_before] = phone_name_after
    
    return map_dic

def get_dic(book,country_name,phone_map):
    country_to_row = {"german":5, "france":6, "uk":7, "italy":8, "spain":9, "finland":10, "czech":11}

    dic = {}
    for i,row in enumerate(book[country_name].values):
        if i > 0:
            brand, phone_type, ram_and_storage = row[0].strip(),row[1].strip(),row[2].strip()
            #这里对手机型号做一下映射，规范同一种手机型号的不同名称
            if phone_type in phone_map:
                phone_type = phone_map[phone_type]           
            # print(phone_type)
            new_country_name = country_name.split("_")[0]
            price_index = country_to_row[new_country_name]
            price = row[price_index]
            
            phone_key = (brand, phone_type, ram_and_storage)
            # choose the lower price
            if phone_key in dic:
                if float(price) < float(dic[phone_key]):
                    dic[phone_key] = price
            else:
                dic[phone_key] = price
    return dic


def create_new_sheet_all(book,file_name,map_excel_path):
    if "all_info_test" in book.sheetnames:
        book.remove(book["all_info_test"])
    ws = book.create_sheet('all_info_test',1)
    ws["A1"],ws["B1"],ws["C1"],ws["D1"],ws["E1"] = "BRAND", "TYPE", "RAM_AND_STORAGE", "TIME", "PRICE" 
    ws["F1"],ws["G1"],ws["H1"],ws["I1"],ws["J1"],ws["K1"],ws["L1"] = "GERMAN","FRANCE","UK","ITALY","SPAIN","FINLAND","CZECH"

    country_list = ["uk_test","german_test","spain_test","france_test"]
    country_upper_to_row = {"GERMAN":5, "FRANCE":6, "UK":7, "ITALY":8, "SPAIN":9, "FINLAND":10, "CZECH":11}

    dic_all = {}
    uk_dic, german_dic, spain_dic, france_dic = {},{},{},{}
    
    phone_map = load_map_dict_from_execl(map_excel_path)
    # print(phone_map)
    #这里的每个dic目的是为了返回每个国家的 <手机型号>:<最低价格> 字典
    uk_dic = get_dic(book,"uk_test",phone_map)
    german_dic = get_dic(book,"german_test",phone_map)
    spain_dic = get_dic(book,"spain_test",phone_map)
    france_dic = get_dic(book,"france_test",phone_map)

    all_phone_type = set()

    for country in country_list:
        country_dic = get_dic(book,country,phone_map)
        for phone_type in country_dic.keys():
            # print(phone_type)
            # phone_type_str = " ".join(phone_type) 
            # all_phone_type.add(phone_type_str)
            all_phone_type.add(phone_type)
    # print(sorted(all_phone_type))
    for x in sorted(all_phone_type):
        # print(x)

    # 表格从第二行开始才是手机和价格
    i = 2
    for p in sorted(all_phone_type):
        brand, phone_type, storage = p
        brand_place = "A" + str(i)
        phone_type_place = "B" + str(i)
        storage_and_memory_place = "C" + str(i)
        # price_place = "G" + str(i)
        
        ws[brand_place] = brand
        ws[phone_type_place] = phone_type
        ws[storage_and_memory_place] = storage
        # ws[price_place] = price

        if p in uk_dic.keys():
            uk_price_place = "H" + str(i)
            uk_price = uk_dic[p]
            ws[uk_price_place] = str(uk_price)
        
        if p in german_dic.keys():
            german_price_place = "F" + str(i)
            german_price = german_dic[p]
            ws[german_price_place] = str(german_price)

        if p in spain_dic.keys():
            spain_price_place = "J" + str(i)
            spain_price = spain_dic[p]
            ws[spain_price_place] = str(spain_price)

        if p in france_dic.keys():
            france_price_place = "G" + str(i)
            france_price = france_dic[p]
            ws[france_price_place] = str(france_price)

        i += 1

    book.save(file_name)


def write_to_final_result(book,file_name):

    price_dic = {}
    info_sheet_name = book["all_info_test"]
    for i,row in enumerate(info_sheet_name.values):
        if i > 0:
            candidate_phone = row[1]
            phone_ram = row[2]
            uni_key = candidate_phone +" " +phone_ram
            german_price,france_price,uk_price,italy_price,spain_price = row[5],row[6],row[7],row[8],row[9]
            price_dic[uni_key] = [german_price,france_price,uk_price,italy_price,spain_price]
    # print(price_dic)

    final_sheet = book["最终结果"]
    for i,row in enumerate(final_sheet.values):
        if i > 1 and row[1]:
            key_phone = row[1].strip().lower()
            ram_ = row[2].strip().split("+")[-1]
            chaxun_key = key_phone + " " + ram_ + "gb"
            # print(chaxun_key)
            if chaxun_key in price_dic.keys():
                german_price_place = "F" + str(i+1)
                spain_price_place = "G" + str(i+1)
                print(price_dic[chaxun_key][0],price_dic[chaxun_key][-1])
                final_sheet[german_price_place] = price_dic[chaxun_key][0]
                final_sheet[spain_price_place] = price_dic[chaxun_key][-1]
    
    book.save(file_name)
    # print("!!!!!!!!")



def run(data_path,file_name,map_excel_path):
    workbook = openpyxl.load_workbook(data_path)
    print(workbook.sheetnames)

    #find the ture number of the excel order
    uk_sheet_name = "英国原数据"
    uk = workbook[uk_sheet_name]
    extract_and_save_uk(workbook,uk,file_name)

    german_sheet_name = "德国原数据"
    german = workbook[german_sheet_name]
    extract_and_save_german(workbook,german,file_name)

    spain_sheet_name = "西班牙原数据"
    spain = workbook[spain_sheet_name]
    extract_and_save_spain(workbook,spain,file_name)  

    france_sheet_name = "法国原数据"
    france = workbook[france_sheet_name]
    extract_and_save_france(workbook,france,file_name) 

    # italy_sheet_name = workbook.sheetnames[5]
    # italy = workbook[italy_sheet_name]
    # extract_and_save_italy(workbook,italy,file_name)  

    create_new_sheet_uk(workbook,uk,file_name)
    create_new_sheet_german(workbook,german,file_name)
    create_new_sheet_spain(workbook,spain,file_name)
    create_new_sheet_france(workbook,france,file_name)

    create_new_sheet_all(workbook,file_name,map_excel_path)
    write_to_final_result(workbook,file_name)

if __name__ == "__main__":
    pwd = os.getcwd()
    file_name = "source_data_and_final_results.xlsx"
    phone_map_excel = "Global_Variable.xlsx"
    xlsx_data_path = os.path.join(pwd,file_name)
    print(xlsx_data_path)
    run(xlsx_data_path,file_name,phone_map_excel)
    # load_map_dict_from_execl("Global_Variable.xlsx")


IndentationError: expected an indented block (Temp/ipykernel_16208/1133951628.py, line 84)